In [1]:
import kagglehub
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [2]:
# --- Função de carregamento de dados ---
def carregar_dataset_kaggle(handle_dataset: str, nome_arquivo_csv: str = None):
    """
    Baixa um dataset do Kaggle Hub e carrega um arquivo CSV específico em um DataFrame.
    Se nome_arquivo_csv não for especificado, carrega o primeiro CSV encontrado.
    """
    print(f"--- Iniciando carregamento de: {handle_dataset} ---")
    try:
        caminho_pasta = kagglehub.dataset_download(handle_dataset)

        # Se um nome de arquivo específico foi fornecido
        if nome_arquivo_csv:
            caminho_completo = os.path.join(caminho_pasta, nome_arquivo_csv)
            if not os.path.exists(caminho_completo):
                # Tenta o caminho sem o 'data/' prefixo, caso o kagglehub já tenha descompactado
                caminho_completo = os.path.join(caminho_pasta, os.path.basename(nome_arquivo_csv))
                if not os.path.exists(caminho_completo):
                    # Tenta o caminho com o 'data/' prefixo, que é o que o usuário usou originalmente
                    caminho_completo = os.path.join(caminho_pasta, 'data', os.path.basename(nome_arquivo_csv))
                    if not os.path.exists(caminho_completo):
                        print(f"ERRO: O arquivo '{nome_arquivo_csv}' não foi encontrado na pasta do dataset.")
                        print(f"Arquivos disponíveis: {os.listdir(caminho_pasta)}")
                        return None
            nome_csv_final = os.path.basename(caminho_completo)
        # Se nenhum nome foi fornecido, usa a lógica original
        else:
            arquivos_csv = [f for f in os.listdir(caminho_pasta) if f.endswith('.csv')]
            if not arquivos_csv:
                print(f"AVISO: Nenhum arquivo .csv encontrado para '{handle_dataset}'.")
                return None
            nome_csv_final = arquivos_csv[0]
            caminho_completo = os.path.join(caminho_pasta, nome_csv_final)

        print(f"Arquivo encontrado: '{nome_csv_final}'. Lendo os dados...")
        df = pd.read_csv(caminho_completo, encoding='utf-8-sig', on_bad_lines='skip')

        print(f"--> Dataset '{handle_dataset}' (arquivo: {nome_csv_final}) carregado com sucesso!\n")
        return df

    except Exception as e:
        print(f"ERRO ao carregar o dataset '{handle_dataset}': {e}\n")
        return None

--- Execução do Notebook Bronze ---

In [3]:
# 1. Carregar dados de estatísticas de produção
df_estatisticas = carregar_dataset_kaggle("nikhilmahajan29/crop-production-statistics-india")

--- Iniciando carregamento de: nikhilmahajan29/crop-production-statistics-india ---
Arquivo encontrado: 'APY.csv'. Lendo os dados...
--> Dataset 'nikhilmahajan29/crop-production-statistics-india' (arquivo: APY.csv) carregado com sucesso!



In [4]:
# 2. Carregar dados de clima (chuva e temperatura)
df_chuva_india = carregar_dataset_kaggle("swarooprangle/indian-agriculture-and-climate-dataset-1961-2018", nome_arquivo_csv="data/rainfall.csv")
df_temp_india = carregar_dataset_kaggle("swarooprangle/indian-agriculture-and-climate-dataset-1961-2018", nome_arquivo_csv="data/temperature.csv")

--- Iniciando carregamento de: swarooprangle/indian-agriculture-and-climate-dataset-1961-2018 ---
Arquivo encontrado: 'rainfall.csv'. Lendo os dados...
--> Dataset 'swarooprangle/indian-agriculture-and-climate-dataset-1961-2018' (arquivo: rainfall.csv) carregado com sucesso!

--- Iniciando carregamento de: swarooprangle/indian-agriculture-and-climate-dataset-1961-2018 ---
Arquivo encontrado: 'temperature.csv'. Lendo os dados...
--> Dataset 'swarooprangle/indian-agriculture-and-climate-dataset-1961-2018' (arquivo: temperature.csv) carregado com sucesso!



In [5]:
# 3. Merge dos dados de clima
if df_chuva_india is None or df_temp_india is None or df_estatisticas is None:
    print("ERRO: Não foi possível carregar um ou mais datasets. Abortando.")
    exit()

In [6]:
df_clima_india = pd.merge(df_chuva_india, df_temp_india, on=['YEAR'])

In [7]:
# 4. Preparação para o merge final e Renomeação de Colunas (MELHORIA 1)
df_producao_final = df_estatisticas.copy()
df_producao_final.rename(columns={'Crop_Year': 'YEAR'}, inplace=True)

In [8]:
# 5. Merge final (Produção + Clima)
df_combinado = pd.merge(
    df_producao_final,
    df_clima_india,
    on='YEAR',
    how='inner'
)

In [ ]:
# 6. Renomear coluna com espaço
df_combinado.rename(columns={'Area ': 'Area'}, inplace=True)

In [10]:
# 7. Renomear colunas climáticas ambíguas (MELHORIA CRÍTICA)
# As colunas de temperatura (em maiúsculas) serão renomeadas para evitar conflito
# com as colunas de precipitação (em minúsculas/capitalizadas)
rename_clima_dict = {
    'JAN-FEB': 'TEMP_JAN_FEB',
    'MAR-MAY': 'TEMP_MAR_MAY',
    'JUN-SEP': 'TEMP_JUN_SEP',
    'OCT-DEC': 'TEMP_OCT_DEC',
    'ANNUAL': 'TEMP_ANNUAL'
}
df_combinado.rename(columns=rename_clima_dict, inplace=True)

In [11]:
print("Merge e Renomeação de Colunas realizado com sucesso!")
print("Dimensões do novo DataFrame:", df_combinado.shape)

Merge e Renomeação de Colunas realizado com sucesso!
Dimensões do novo DataFrame: (325753, 30)


In [12]:
# 8. Camada Bronze
os.makedirs('data/bronze', exist_ok=True)
os.makedirs('data/silver', exist_ok=True)
os.makedirs('data/gold', exist_ok=True)

In [13]:
df_raw = df_combinado.copy()

In [14]:
print(f"Dados carregados: {df_raw.shape[0]} linhas, {df_raw.shape[1]} colunas")

Dados carregados: 325753 linhas, 30 colunas


In [15]:
# Adicionar informações de quando os dados foram carregados
df_raw['data_ingestao'] = datetime.now()

In [16]:
# Salvar na camada Bronze
df_raw.to_csv('data/bronze/dados_brutos.csv', index=False)
print("Dados salvos na camada Bronze")

Dados salvos na camada Bronze
